<a href="https://colab.research.google.com/github/alimadelshin/competitive/blob/main/Competitive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing


In [1]:
!git clone https://github.com/IlyaGusev/purano

Cloning into 'purano'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (251/251), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 987 (delta 118), reused 180 (delta 70), pack-reused 736
Receiving objects: 100% (987/987), 1.24 MiB | 16.97 MiB/s, done.
Resolving deltas: 100% (528/528), done.


In [2]:
%cd purano

/content/purano


In [3]:
!pip install -r requirements.txt
!pip install pip install sentence-transformers

In [4]:
!mkdir models/lang_detect
!mkdir models/cat_detect
!wget https://www.dropbox.com/s/hoapmnvqlknmu6v/lang_detect_v10.ftz -O models/lang_detect/lang_detect_v10.ftz
!wget https://www.dropbox.com/s/23x35wuet280eh6/ru_cat_v5.ftz -O models/cat_detect/ru_cat_v5.ftz

--2021-02-27 19:06:45--  https://www.dropbox.com/s/hoapmnvqlknmu6v/lang_detect_v10.ftz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/hoapmnvqlknmu6v/lang_detect_v10.ftz [following]
--2021-02-27 19:06:46--  https://www.dropbox.com/s/raw/hoapmnvqlknmu6v/lang_detect_v10.ftz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc36feee1a016d62199751e5ce85.dl.dropboxusercontent.com/cd/0/inline/BJyZHgWLXsSyxb1rcji44J7nmoDQ2krWy-6sSegNlDJQCN0uqCWxRObZoX-Fd0mlXMgT9r2sFar5zDwXces3so6RPWbpEMncuc_x6E6YyDlbq7zT8njRBDymBDqtrtMPnuQ/file# [following]
--2021-02-27 19:06:46--  https://uc36feee1a016d62199751e5ce85.dl.dropboxusercontent.com/cd/0/inline/BJyZHgWLXsSyxb1rcji44J7nmoDQ2krWy-6sSegNlDJQCN0uqCWxRObZoX-Fd0mlXMgT9r2sFar5zDwXces3so6RPW

In [5]:
# May 25 (training)
!rm -rf data
!mkdir -p data
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0525.tar.gz -O - | tar -xz && cd ../

--2021-02-27 19:06:59--  https://data-static.usercontent.dev/DataClusteringDataset0525.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145541247 (139M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>] 138.80M  16.6MB/s    in 8.6s    

2021-02-27 19:07:08 (16.1 MB/s) - written to stdout [145541247/145541247]



In [6]:
%%writefile configs/cleaner.jsonnet
{
    "lang_detect_model_path": "models/lang_detect/lang_detect_v10.ftz",
    "cat_detect_model_path": "models/cat_detect/ru_cat_v5.ftz",
    "is_lower": true,
    "is_russian_only": true,
    "is_news_only": true
}

Overwriting configs/cleaner.jsonnet


In [7]:
!rm -f 0525_parsed.db
!python3 -m purano.run_parse --inputs "data/20200525" --output-file 0525_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet

Parsed 10000 documents
Parsed 20000 documents
Parsed 30000 documents
Parsed 40000 documents
Parsed 50000 documents
Parsed 60000 documents
Parsed 70000 documents
Parsed 80000 documents
19380 will be saved
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19380 entries, 3112 to 7997
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype                    
---  ------         --------------  -----                    
 0   url            19380 non-null  object                   
 1   host           19380 non-null  object                   
 2   title          19380 non-null  object                   
 3   text           19380 non-null  object                   
 4   date           19380 non-null  datetime64[ns, tzlocal()]
 5   patched_title  19380 non-null  object                   
 6   patched_text   19380 non-null  object                   
 7   category       19380 non-null  object                   
dtypes: datetime64[ns, tzlocal()](1), object(7)
memory usage: 1.3+

In [17]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from purano.models import Document

def parse_db(file_name):
    db_engine = "sqlite:///{}".format(file_name)
    engine = create_engine(db_engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    query = session.query(Document)
    docs = query.all()
    records = []
    for doc in docs:
        records.append({
            "title": doc.title,
            "text": doc.text,
            "url": doc.url,
            "host": doc.host,
            "timestamp": doc.date,
            "patched_title": doc.patched_title,
            "patched_text": doc.patched_text
        })
    return records

records = parse_db("0525_parsed.db")


In [9]:
# Clustering training markup
!wget https://www.dropbox.com/s/8lu6dw8zcrn840j/ru_clustering_0525_urls.tsv

--2021-02-27 19:09:09--  https://www.dropbox.com/s/8lu6dw8zcrn840j/ru_clustering_0525_urls.tsv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8lu6dw8zcrn840j/ru_clustering_0525_urls.tsv [following]
--2021-02-27 19:09:09--  https://www.dropbox.com/s/raw/8lu6dw8zcrn840j/ru_clustering_0525_urls.tsv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc67ceb7a929042a32efc1811e78.dl.dropboxusercontent.com/cd/0/inline/BJw2fe8OYXpTuFWYZlsuymf6JpkwSVUZHHRejnn7qRYbMlY7rvQtUfJZLcET5_xz3z_ki1ZJVnCbL7j4meCTtT7B8c1iMOdlp8plD_PyK_6t_WSF0bx-XdDeoHamDXSY6Vk/file# [following]
--2021-02-27 19:09:10--  https://uc67ceb7a929042a32efc1811e78.dl.dropboxusercontent.com/cd/0/inline/BJw2fe8OYXpTuFWYZlsuymf6JpkwSVUZHHRejnn7qRYbMlY7rvQtUfJZLcET5_xz3z

# Evaluation


In [41]:
import random
from purano.io import read_markup_tsv
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

    def __init__(self, x, y):
        self.len = len(x)
        self.data_x = x
        self.data_y = y
    
    def __getitem__(self, index):
        return (self.data_x[index], self.data_y[index])
    
    def __len__(self):
        return self.len


#Выбираем только "OK" разметки и делим данные на train/test
def pick_ok_samples(markup, train_percent=0.85):
    temp = []
    for mrkp in markup:
      if mrkp['quality'] == 'OK':
        temp.append(mrkp)
    
    count_samples = len(temp)
    shuffled_ids = list(range(count_samples))
    random.shuffle(shuffled_ids)
    train_ids = round(count_samples * train_percent)

    train_markup, test_markup = [], []

    for i, id in enumerate(shuffled_ids):
      if i < train_ids:
        train_markup.append(temp[id])
      else:
        test_markup.append(temp[id])
  
    return train_markup, test_markup


def get_train_data(markup, records):
    input_samples, pos_samples = [], []

    for mrkp in markup:
        first_url, second_url, quality = mrkp.values()
        if quality == 'OK':
            input_samples.append(url2record[first_url]['title'] + ' ' + url2record[first_url]['text'])
            pos_samples.append(url2record[second_url]['title'] + ' ' + url2record[second_url]['text'])
    
    return input_samples, pos_samples

def get_test_records(markup, records):
    test_records = []
    for record in records:
      for mrkp in markup:
        first_url, second_url, quality = mrkp.values()
        if record['url'] == first_url or record['url'] == second_url:
            test_records.append(record)
    
    return test_records


url2record = dict()

for i, record in enumerate(records):
    url2record[record["url"]] = record


markup = read_markup_tsv("ru_clustering_0525_urls.tsv")
train_markup, test_markup = pick_ok_samples(markup)
input_samples, pos_samples = get_train_data(train_markup, records)
test_records = get_test_records(test_markup, records)


In [11]:
from collections import Counter
from statistics import median, mean
from sklearn.cluster import AgglomerativeClustering
from purano.clusterer.metrics import calc_metrics

def get_quality(markup, embeds, records, dist_threshold, print_result=False):
    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=dist_threshold,
        linkage="average",
        affinity="cosine"
    )

    clustering_model.fit(embeds)
    labels = clustering_model.labels_

    idx2url = dict()
    url2record = dict()
    for i, record in enumerate(records):
        idx2url[i] = record["url"]
        url2record[record["url"]] = record

    url2label = dict()
    for i, label in enumerate(labels):
        url2label[idx2url[i]] = label
        
    metrics = calc_metrics(markup, url2record, url2label)[0]
    if print_result:
        print()
        print("Accuracy: {:.1f}".format(metrics["accuracy"] * 100.0))
        print("Positives Recall: {:.1f}".format(metrics["1"]["recall"] * 100.0))
        print("Positives Precision: {:.1f}".format(metrics["1"]["precision"] * 100.0))
        print("Positives F1: {:.1f}".format(metrics["1"]["f1-score"] * 100.0))
        print("Distance: ", dist_threshold)
        sizes = list(Counter(labels).values())
        print("Max cluster size: ", max(sizes))
        print("Median cluster size: ", median(sizes))
        print("Avg cluster size: {:.2f}".format(mean(sizes)))
        return
    return metrics["1"]["f1-score"]

# USE-only

In [21]:
import sentence_transformers

use = sentence_transformers.SentenceTransformer('distiluse-base-multilingual-cased-v2')


100%|██████████| 504M/504M [00:22<00:00, 22.6MB/s]


In [28]:
import numpy as np
from tqdm.notebook import tqdm

def use_get_embedding(text, model):
    return model.encode([text])

def use_records_to_embeds(records, model):
    embeddings = np.zeros((len(records), 512))
    for i, record in tqdm(enumerate(records)):
        embeddings[i] = use_get_embedding(record["title"] + " " + record["text"], model)
    return embeddings

In [48]:
train_use_embeddings = use_records_to_embeds(test_records, use)

TypeError: ignored

In [39]:
get_quality(test_markup, train_use_embeddings, test_records, 0.37, print_result=True)


Accuracy: 87.6
Positives Recall: 87.6
Positives Precision: 100.0
Positives F1: 93.4
Distance:  0.37
Max cluster size:  22
Median cluster size:  2.0
Avg cluster size: 2.70


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# OurModel

In [42]:
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):

    def __init__(self, x, y):
        self.len = len(x)
        self.data_x = x
        self.data_y = y
    
    def __getitem__(self, index):
        return (self.data_x[index], self.data_y[index])
    
    def __len__(self):
        return self.len


dataset = CustomDataset(input_samples, pos_samples)

In [ ]:
import sentence_transformers

use = sentence_transformers.SentenceTransformer('distiluse-base-multilingual-cased-v2')


In [43]:
import math

import torch
from torch import nn

class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


class FeedForwardTop(nn.Module):
    def __init__(self, dropout = 0.05):
        super().__init__()
        self.f1 = nn.Linear(512, 512)
        self.dropout = nn.Dropout(dropout) 
        self.gelu = GELU()
        self.f2 = nn.Linear(512, 512)

    
    def forward(self, x):
        x = self.dropout(self.gelu(self.f1(x)))
        x = self.f2(x)

        return x


In [45]:
from tqdm.notebook import tqdm

def use_get_embedding(text, model, ff, device):
    return ff(torch.tensor(model.encode([text]))).detach().numpy()

def use_records_to_embeds(records, model, ff, device):
    embeddings = np.zeros((len(records), 512))
    for i, record in tqdm(enumerate(records)):
        embeddings[i] = use_get_embedding(record["title"] + " " + record["text"], model, ff, device)
    return embeddings

In [50]:
import torch.nn.functional as F
import numpy as np
from tqdm.notebook import tqdm

torch.manual_seed(0)

batch_size = 32
lr = 0.01


device = 'cpu'
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = FeedForwardTop().to(device)

for p in net.parameters():
    if p.dim() > 1:
        torch.nn.init.xavier_uniform_(p)


optimizer = torch.optim.Adam(net.parameters(), lr=lr)

train_loader = DataLoader(dataset, batch_size=batch_size)

EPOCHES = 3

total_loss = 0


for epoch in range(EPOCHES):
    net.train()
    for i, (input_samples, pos_samples) in enumerate(tqdm(train_loader)):
        optimizer.zero_grad()

        q_vectors = torch.tensor(use.encode(input_samples)).to(device)
        ctx_vectors = torch.tensor(use.encode(pos_samples)).to(device)

        q_vectors = net(q_vectors)
        q_vectors = F.normalize(q_vectors, p=2, dim = -1)

        ctx_vectors = net(ctx_vectors)
        ctx_vectors = F.normalize(ctx_vectors, p=2, dim = -1)

        scores = torch.matmul(q_vectors, torch.transpose(ctx_vectors, 0, 1))

        if len(q_vectors.size()) > 1:
            q_num = q_vectors.size(0)
            scores = scores.view(q_num, -1)
        
        softmax_scores = F.log_softmax(scores, dim=1)
        
        pos_idx_per_question = torch.tensor(list(range(q_vectors.size(0)))).to(device)


        loss = F.nll_loss(softmax_scores,
                   pos_idx_per_question,
                   reduction='mean')

        loss.backward()
        optimizer.step()
        
        total_loss += loss.data

        if (i + 1) % 45 == 0:
          print(f'epoch: {(epoch + 1)} batch_idx: {(i + 1)} loss: {total_loss/45}')
          total_loss = 0
      

    net.eval()
    embeds = use_records_to_embeds(test_records, use, net)



KeyboardInterrupt: ignored

In [49]:
get_quality(test_markup, embeds, test_records, dist_threshold=0.37, print_result=True)

NameError: ignored